In [3]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [1]:
import pandas as pd
import numpy as np
import tqdm
pd.set_option('display.max_rows', 1000)
from bs4 import BeautifulSoup

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Ashik/Thesis/'
raw_data = pd.read_csv(f'{path}data/dataset-2.csv')
raw_data.shape


import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import GPT2TokenizerFast
import nltk

nltk.download('stopwords')
nltk.download('punkt')

CheckPoint = "gpt2"
ds = DatasetDict()

stemmer = PorterStemmer()

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    cleaned_text = ' '.join(stemmed_tokens)
    return cleaned_text


raw_data['description'] = raw_data['description'].apply(clean_text)
df = raw_data

df['labels'] = df['category']

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=1234 )
train_df.shape, test_df.shape


hf_train = Dataset.from_pandas(train_df, preserve_index=False)
hf_val = Dataset.from_pandas(test_df, preserve_index=False)

hf_train = hf_train.class_encode_column("labels")
hf_val = hf_val.class_encode_column("labels")

print(hf_train.features)


ds['train'] = hf_train
ds['validation'] = hf_val



tokenizer = GPT2TokenizerFast.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
  return tokenizer(example['description'], padding=True, truncation=True)


tokenized_datasets = ds.map(tokenize_function, batched=True)
print(tokenized_datasets)


model = AutoModelForSequenceClassification.from_pretrained("openai-community/gpt2", num_labels=9).to('cuda')

training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size= 1,
    per_device_eval_batch_size= 1,
    num_train_epochs = 5,
    learning_rate = 2e-5,
    weight_decay = 0.01)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    #data_collator=data_collator,
    tokenizer=tokenizer,
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Casting to class labels:   0%|          | 0/1688 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/423 [00:00<?, ? examples/s]

{'collection': Value(dtype='string', id=None), 'description': Value(dtype='string', id=None), 'category': Value(dtype='string', id=None), 'labels': ClassLabel(names=['arts', 'avatar', 'collectibles', 'games', 'memberships'], id=None)}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1688 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['collection', 'description', 'category', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1688
    })
    validation: Dataset({
        features: ['collection', 'description', 'category', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 423
    })
})


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [2]:
from sklearn import preprocessing
from sklearn import metrics

trainer.train()
trainer.save_model(f"path/model_checkpoint/{CheckPoint}")

Step,Training Loss
500,1.479300
1000,1.370100
1500,1.107300
2000,0.709300
2500,0.713800
3000,0.740600
3500,0.654700
4000,0.504000
4500,0.469600


Checkpoint destination directory test-trainer/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Step,Training Loss
500,1.479300
1000,1.370100
1500,1.107300
2000,0.709300
2500,0.713800
3000,0.740600
3500,0.654700
4000,0.504000
4500,0.469600
5000,0.397500


In [3]:
predictions = trainer.predict(tokenized_datasets['validation'])

test_df['prediction_labels'] = predictions.label_ids
test_df['prediction_label_argmax'] = [np.argmax(prediction) for prediction in predictions.predictions]

print(test_df.shape)

le = preprocessing.LabelEncoder()
test_df['actual_label'] = le.fit_transform(test_df.labels.values)

(423, 6)


In [4]:
num_correct_labels = (test_df['prediction_labels'] == test_df['prediction_label_argmax']).sum()

acc = metrics.accuracy_score(test_df.actual_label, test_df.prediction_label_argmax)
f1 = metrics.f1_score(test_df.actual_label, test_df.prediction_label_argmax, average='macro')
precision = metrics.precision_score(test_df.actual_label, test_df.prediction_label_argmax, average='macro')
recall =  metrics.recall_score(test_df.actual_label, test_df.prediction_label_argmax, average='macro')

print(f"tot samples                : {len(test_df)}")
print(f"num_correct_labels         : {num_correct_labels}")
print(f"accuracy                   : {acc}")
print(f"f1_score                   : {f1}")
print(f"recall_score               : {recall}")
print(f"precision_score            : {precision}")

tot samples                : 423
num_correct_labels         : 382
accuracy                   : 0.9030732860520094
f1_score                   : 0.9037925794483478
recall_score               : 0.90809718659196
precision_score            : 0.9002223410576352


In [5]:
data = {
      'tot_samples': len(test_df),
        'is_correct': num_correct_labels,
        'accuracy': acc,
        'f1_score': f1,
        'recall_score': recall,
        'precision_score': precision,
        'actual_label': test_df.actual_label,
        'prediction_label_argmax': test_df.prediction_label_argmax,
}
df = pd.DataFrame(data)
df

,tot_samples,is_correct,accuracy,f1_score,recall_score,precision_score,actual_label,prediction_label_argmax
836,423,382,0.903073,0.903793,0.908097,0.900222,4,4
1189,423,382,0.903073,0.903793,0.908097,0.900222,3,3
1290,423,382,0.903073,0.903793,0.908097,0.900222,3,3
1286,423,382,0.903073,0.903793,0.908097,0.900222,3,3
910,423,382,0.903073,0.903793,0.908097,0.900222,4,4
602,423,382,0.903073,0.903793,0.908097,0.900222,4,4
1325,423,382,0.903073,0.903793,0.908097,0.900222,3,3
1494,423,382,0.903073,0.903793,0.908097,0.900222,3,3
809,423,382,0.903073,0.903793,0.908097,0.900222,4,4
1714,423,382,0.903073,0.903793,0.908097,0.900222,2,2


In [6]:
df.to_csv(f'{path}results/approach-2-gpt2.csv', index = False)

In [7]:
import pandas as pd
path = '/content/drive/MyDrive/Ashik/Thesis/'

df = pd.read_csv(f'{path}results/approach-2-gpt2.csv')
actual_labels = df['actual_label']
predicted_labels = df['prediction_label_argmax']
classes = ['arts', 'avatar', 'collectibles', 'games', 'memberships']

In [8]:
import plotly.graph_objs as go
import plotly.io as pio
import pandas as pd

def plot_confusion_matrix(true, pred, labels):
    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(true, pred, normalize='true')
    cm_df = pd.DataFrame(confusion_matrix,
                         index=labels,
                         columns=labels)

    # Reverse the order of the labels for the y-axis
    reversed_labels = labels[::-1]

    columns = cm_df.columns
    columns = [i.capitalize() for i in columns]

    reversed_labels = [i.capitalize() for i in reversed_labels]

    fig = go.Figure(data=go.Heatmap(z=cm_df.values[::-1],  # Reverse the rows
                                     x=columns,
                                     y=reversed_labels,  # Use reversed labels for y-axis
                                     text=cm_df.values[::-1],  # Reverse the rows for text as well
                                     texttemplate="%{text:.1%}",
                                     textfont=dict(size=12),  # Set the font size
                                     colorbar=dict(title='Percentage', tickformat='.1%'),
                                     hoverinfo='skip',
                                     colorscale='Greens'))  # Change the colorscale here

    fig.update_layout(
                      xaxis_title=dict(text='Predicted Label', font=dict(size=15, family='Palatino')),  # Capitalize first letter
                      yaxis_title=dict(text='True Label', font=dict(size=15, family='Palatino')),  # Capitalize first letter
                      font=dict(family='Palatino'),
                      margin=dict(l=0, r=0, t=50, b=50),
                      xaxis=dict(title_standoff=30),
                      yaxis=dict(title_standoff=30))  # Adjust the position of the y-axis title

    # Increase font size for x-axis labels (columns) and y-axis labels (reversed_labels)
    fig.update_xaxes(tickfont=dict(size=12, family='Palatino'))
    fig.update_yaxes(tickfont=dict(size=12, family='Palatino'))

    pio.write_image(fig, 'approach-1-gpt2-cm.jpeg', format='jpeg', scale=10)

    fig.show()


In [9]:
plot_confusion_matrix(actual_labels,predicted_labels,['arts', 'avatar', 'collectibles', 'games', 'memberships'])